In [1]:
import bct
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import pingouin
import matplotlib.pyplot as plt
import warnings
sys.path.append("/scratch/connectome/TVB/TVB_RESEARCH")
sys.path
import utils #now imports becasue it looks at the last added sys path

import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
import argparse


#parser = argparse.ArgumentParser(description = "argparse")
#
#parser.add_argument('--threshold', type = float, help = "threshold to use")
#
#args = parser.parse_args()

class arg():
    def __init__(self):
        self.threshold = 0.1
args = arg()   
data_dir = '/storage/bigdata/ABCD/TVB/data'

####load dataset and stuff 
dataset = utils.load_data.ABCDDataset(data_dir)  #이제 된다
subjects = dataset.list_subjects()


start = time.time()
warnings.filterwarnings('once') #여기서는 한번만 나오게 하기 (여러번 돌아가면서 계속 뜨는 것은 싫다)

###how much to keep
sample_size = len(subjects) #if you wanna run all
threshold = args.threshold

##scalar meausres initialization
sample_sub = dataset.load_sc(subjects[0])
data, names = utils.conn_measures.compute_bct_UW(sample_sub, threshold = 0.3).scalar_properties()


num_sub = len(subjects) #8238
num_measures = len(names)
scalar_bct_results = np.empty((min(num_sub,sample_size),num_measures))
scalar_bct_results[:] = np.nan


###SC + SCALAR (vector, matrix는 할지말지, 그리고 한다면 언제할지는 모르겠다)
warnings.filterwarnings("ignore")
tqdm_loop = tqdm(subjects[1445+0:1447+30]) #tqdm_loop이라는 tqdm object로 만들기 => 나중에 descrioption을 둘거라서
fail_count = 0
for i,sub in enumerate(tqdm_loop):#enumerate(subjects[:sample_size]):#tqdm(subjects[:sample_size])):
    sub_sc = dataset.load_sc(sub)
    #do FC too!!!
        
    #trying to load the thing 
        #utils.conn_measuers 그거 initiailize할때, this has "check usaiblity" built-in (in __init__) 
        #threfore, when the matrix_loaded fails, it means the matrix is not usable
    try :
        matrix_loaded = utils.conn_measures.compute_bct_UW(sub_sc, threshold=threshold)
    except : 
        fail_count += 1
        tqdm_loop.set_postfix_str("sub-{} failed! || {}th fail".format(sub,fail_count))
        continue

    data, names = matrix_loaded.scalar_properties()
    scalar_bct_results[i] = np.array(list(data.values()))

total_scalar_results = pd.DataFrame(scalar_bct_results, index = subjects[:sample_size], columns = names)
end = time.time()
print("run_time(s) :", end-start)


BCT_result_save_dir = "/storage/bigdata/ABCD/TVB/BCT_results/SC_ses_2/"

total_scalar_results.to_csv(BCT_result_save_dir+"scalar_results_threshold_{}.csv".format(threshold))
pd.read_csv(BCT_result_save_dir+"scalar_results_threshold_{}.csv".format(threshold), index_col=0) #use the first column as the rows
os.listdir(BCT_result_save_dir)

/home/connectome/dyhan316/.local/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


IMPORTED


/home/connectome/dyhan316/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:43<00:00,  3.22s/it, sub-NDARINV8J6JMUXE failed! || 19th fail]

run_time(s) : 103.96608996391296


['scalar_results_threshold_0.1.csv']

In [2]:
print(count)

NameError: name 'count' is not defined

In [3]:
print(scalar_bct_results[:10])
#print(scalar_bct_results.shape[3:10])
#pd.read_csv(BCT_result_save_dir+"scalar_results_threshold_{}.csv".format(threshold), index_col=0) #use the first column as the rows

[[ 6.23955553e-02  6.65323609e-02  7.01519352e-02  1.04776235e+02
   2.25533864e+01  6.65323609e-02  5.18369775e+01  9.07542255e+01
   9.16857778e-01  5.90648245e-01 -1.01079889e-01]
 [ 6.05985248e-02  5.85524563e-02 -1.51081885e-02  9.70514073e+01
   2.64040148e+01  5.85524563e-02  4.18586185e+01  8.27393973e+01
   9.32565388e-01  6.43401172e-01 -9.56939376e-02]
 [            nan             nan             nan             nan
              nan             nan             nan             nan
              nan             nan             nan]
 [            nan             nan             nan             nan
              nan             nan             nan             nan
              nan             nan             nan]
 [            nan             nan             nan             nan
              nan             nan             nan             nan
              nan             nan             nan]
 [ 5.72918348e-02  5.50447577e-02  1.64806929e-02  9.22843137e+01
   2.73457986e+01  

In [ ]:
a = 0
while a !=5:
    a = a+1
    print(a)
    if a > 2:
        break
        

In [ ]:
scalar_bct_results
#1447 : error이 난다

In [ ]:
print(scalar_bct_results[:10])

In [ ]:
pd.

In [ ]:
class try_class():
    def __init__(self, x,y):
        self.x = x
        self.y = y
        
        print(self.check())
        if self.check() == None:
            raise Exception("error")
        print(x+y)
        
    def check(self):
        if self.x+self.y==3:
            return True
        else :
            return None
a = try_class(3,1)

    

In [ ]:
for i,sub in enumerate(subjects[:sample_size]): #tqdm(subjects[:sample_size])): #tqdm not used becuase if loop breaks during tqdm tqdm breaks
    sub_sc = dataset.load_sc(sub)
    #do FC too!!!
    matrix_loaded = utils.conn_measures.compute_bct_UW(sub_sc, threshold=threshold)
    if matrix_loaded.usability_check_density_based() != True:
        print("{}failed".format(sub))
        continue #if usability check is not satisfied, skip this iteration
    data, names = matrix_loaded.scalar_properties()
    scalar_bct_results[i] = np.array(list(data.values()))



In [ ]:
count = 0
modular_structures = np.zeros(5)
matrix_loaded = utils.conn_measures.compute_bct_UW(dataset.load_sc(subjects[4]), threshold=threshold)
print(matrix_loaded.usability_check_density_based())
while modular_structures.max() != 5:
    modular_structures, modularities = bct.community_louvain(matrix_loaded.mat)
    count+=1
    print(count)
    if count > 1000: #MUST BE CHANGED
        if (np.max(modular_structures) == 6) or (np.max(modular_structures)==4):
            break
print(modular_structures)

In [ ]:
#아!!! 4번쩨가 안되는 이유가, __init__예가 하면서 돌려야하는 것이 있는데, 이게 그 작동하면 안된느 것이기 때문!
#따라서, __init__에서 정의하기 전에, 먼저 check usaiblity본 후에, usability가 check되었을때만 밑에 하도록 하기
#단, 만약 usability check 통과해도 작동을 안하는 것이 된다면,... => 이거는 조금있다가 더 생각해보자

In [ ]:
import tqdm 
import time
for i, name in enumerate(tqdm.tqdm(["ahha","hehe", "hoho"])):
    
    print(i)
    if i==1:
        continue
    time.sleep(2)
    print("haha{}".format(i))

In [ ]:
a = np.arange(10*10).